## Download the dataset



In [5]:
!git clone https://github.com/mttgdd/RobotCarDataset-Scraper.git

Cloning into 'RobotCarDataset-Scraper'...
remote: Enumerating objects: 196, done.
remote: Total 196 (delta 0), reused 0 (delta 0), pack-reused 196
Receiving objects: 100% (196/196), 46.33 KiB | 558.00 KiB/s, done.
Resolving deltas: 100% (108/108), done.


In [129]:
!python RobotCarDataset-Scraper/scrape_mrgdatashare.py --downloads_dir Downloads --datasets_file RobotCarDataset-Scraper/demo.csv --username zirui --password 2251598zzrrywD

ScrapeMRGDatashare is starting...
reading datasets_file: /content/RobotCarDataset-Scraper/demo.csv
got num_datasets: 1
got csrf_middleware_token: WgcqdeE3PDEgBsD4LAlgl5j6CFj6fnbqaGk8StaGixBxGB9neIYwvSjnNiob1H2x
Logged in!
num_chunks_in_period: 0, period_seconds: 10
requesting file_url: http://mrgdatashare.robots.ox.ac.uk:80/download/?filename=datasets/2015-05-19-14-06-38/2015-05-19-14-06-38_tags.tar
downloading local_file_path: /content/Downloads/2015-05-19-14-06-38_tags.tar
100% 10.0/10.0 [00:00<00:00, 29.8kKB/s]
unzipping local_file_path: /content/Downloads/2015-05-19-14-06-38_tags.tar
requesting file_url: http://mrgdatashare.robots.ox.ac.uk:80/download/?filename=datasets/2015-05-19-14-06-38/2015-05-19-14-06-38_stereo_left_01.tar
downloading local_file_path: /content/Downloads/2015-05-19-14-06-38_stereo_left_01.tar
100% 4.52M/4.52M [01:24<00:00, 53.6kKB/s]
unzipping local_file_path: /content/Downloads/2015-05-19-14-06-38_stereo_left_01.tar
requesting file_url: http://mrgdatashare.rob

## Extract descriptor



In [9]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
import numpy as np
import cv2
from PIL import Image


model_file = 'alexnet_places365.pth.tar'
if not os.access(model_file, os.W_OK):
    os.system('wget http://places2.csail.mit.edu/models_places365/' + model_file)


# from functools import partial
# import pickle
# pickle.load = partial(pickle.load, encoding="latin1")
# pickle.Unpickler = partial(pickle.Unpickler, encoding="latin1")
# model = torch.load(model_file, map_location=lambda storage, loc: storage, pickle_module=pickle)

model = models.alexnet(num_classes=365)
checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [0]:
import torch.nn as nn
def returnTF():
# load the image transformer
    tf = trn.Compose([
        trn.Resize((224,224)),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    return tf

def myforward(model, x):
    m1 = nn.Sequential(*list(model.children())[:-1])
    m2 = nn.Sequential(*list(model.children())[2][:2])
    feature = m1(x)
    feature = torch.flatten(feature, start_dim=0, end_dim=-1)
    feature = m2(feature)
    feature = feature.view(x.size(0), -1)
    return feature


# forward image
tf = returnTF()

# read image

img = cv2.imread('/content/Downloads/2014-12-09-13-21-02/stereo/left/1418132549656281.png',cv2.IMREAD_GRAYSCALE)
img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_GRAY2RGB))

input_img = V(tf(img).unsqueeze(0))

# forward pass
logit = model.forward(input_img)
output = myforward(model, input_img)

In [116]:
x = "123.png"
savepath = "/content/2014-12-09-13-21-02/"
savepath+x[:-4]+'.mat'

'/content/2014-12-09-13-21-02/123.mat'

In [0]:
import os
filepath = "/content/Downloads/2015-05-19-14-06-38/stereo/left/"
savepath = "/content/2015-05-19-14-06-38/"
 
for filename in os.listdir(filepath):
    img = cv2.imread(filepath+filename,cv2.IMREAD_GRAYSCALE)
    img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_GRAY2RGB))
    input_img = V(tf(img).unsqueeze(0))
    output = myforward(model, input_img)
    x = output.data.numpy()
    io.savemat(savepath+filename[:-4]+'.mat', {'fc6': x}) 


In [136]:
!zip -r 2014-12-10-18-10-50.zip /content/2014-12-10-18-10-50/
!zip -r 2014-12-16-09-14-09.zip /content/2014-12-16-09-14-09/
!zip -r 2015-02-03-08-45-10.zip /content/2015-02-03-08-45-10/
!zip -r 2015-05-19-14-06-38.zip /content/2015-05-19-14-06-38/

Streaming output truncated to the last 5000 lines.
  adding: content/2015-05-19-14-06-38/1432044663009195.mat (deflated 8%)
  adding: content/2015-05-19-14-06-38/1432044765433037.mat (deflated 8%)
  adding: content/2015-05-19-14-06-38/1432044678132318.mat (deflated 8%)
  adding: content/2015-05-19-14-06-38/1432044754621966.mat (deflated 8%)
  adding: content/2015-05-19-14-06-38/1432044571146670.mat (deflated 8%)
  adding: content/2015-05-19-14-06-38/1432044814239010.mat (deflated 8%)
  adding: content/2015-05-19-14-06-38/1432044783243101.mat (deflated 8%)
  adding: content/2015-05-19-14-06-38/1432044540025815.mat (deflated 8%)
  adding: content/2015-05-19-14-06-38/1432044420291939.mat (deflated 8%)
  adding: content/2015-05-19-14-06-38/1432044651760695.mat (deflated 8%)
  adding: content/2015-05-19-14-06-38/1432044526777616.mat (deflated 8%)
  adding: content/2015-05-19-14-06-38/1432044725625894.mat (deflated 8%)
  adding: content/2015-05-19-14-06-38/1432044577645758.mat (deflated 8%)


In [137]:
from google.colab import files
files.download('2014-12-10-18-10-50.zip')
files.download('2014-12-16-09-14-09.zip')
files.download('2015-02-03-08-45-10.zip')
files.download('2015-05-19-14-06-38.zip')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 39706, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [138]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!rm -rf /content/Downloads/2014-12-09-13-21-02